In [1]:
import string
from copy import deepcopy as copy

from IPython.display import Image, clear_output, display
from Libs.Beholder import *
from Libs.MonkeyRunner import MonkeyRunner
from PIL import Image as pilImage
from tqdm.notebook import tqdm

In [2]:
a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(
    Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image", enabled=True)
)

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="gray_letters",
        from_layer="gray",
        x=304,
        y=1632,
        w=850,
        h=860,
        enabled=True,
    )
)

In [3]:
from pathlib import Path

import requests

w = Path("words.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://raw.githubusercontent.com/MariaLuo826/english_words/master/words_alpha.txt"
        ).content
    )
words = [l.upper() for l in w.read_text().splitlines()]
len(words)

370105

In [4]:
# Top words
w = Path("1ktopwords.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://gist.githubusercontent.com/deekayen/4148741/raw/98d35708fa344717d8eee15d11987de6c8e26d7d/1-1000.txt"
        ).content
    )
topwords = [l.upper() for l in w.read_text().splitlines()]
len(topwords)

1000

In [5]:
# wiki words
w = Path("wikiwords.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://gist.githubusercontent.com/h3xx/1976236/raw/bbabb412261386673eff521dddbe1dc815373b1d/wiki-100k.txt"
        ).content
    )
wikiwords = list(sorted(list(set([l.upper() for l in w.read_text().splitlines()]))))
len(wikiwords)

70926

In [6]:
# h3x words
w = Path("1ktopwords.lst")
if not w.exists():
    w.write_bytes(requests.get("").content)
topwords = [l.upper() for l in w.read_text().splitlines()]
len(topwords)

1000

In [7]:
import shutil

for l in string.ascii_uppercase:
    p1 = Path(f"./wordtrip_templates/{l}/")
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        shutil.move(p, p1)

In [8]:
a.matchers = {}
for l in string.ascii_uppercase:
    p = Path(f"./wordtrip_templates/{l}")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l,
                layer="gray_letters",
                batch_folder=str(p),
                threshhold=0.9,
            )
        )

for l in ["claim_backpack"]:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l,
                layer="gray",
                filename=str(p),
                threshhold=0.9,
            )
        )
for p in Path("./wordtrip_templates").glob("close_*.png"):
    print(p)
    a.addMatcher(
        Beholder_Image_Matcher(
            name=p.stem,
            layer="gray",
            filename=str(p),
            threshhold=0.7,
        )
    )
"""
for l in ["white_x"]:
    print(f"Loading masked {l}")
    p = Path(f"./wordtrip_templates/{l}.png")
    a.addMatcher(
        Beholder_Image_Matcher(
            name=l,
            layer="gray",
            filename=str(p),
            mask_filename=f"./wordtrip_templates/{l}_mask.png",
            threshhold=0.9,
        )
    )"""

Loading A
Loading B
Loading C
Loading D
Loading E
Loading F
Loading G
Loading H
Loading I
Loading J
Loading K
Loading L
Loading M
Loading N
Loading O
Loading P
Loading Q
Loading R
Loading S
Loading T
Loading U
Loading V
Loading W
Loading X
Loading Y
Loading Z


'\nfor l in ["white_x"]:\n    print(f"Loading masked {l}")\n    p = Path(f"./wordtrip_templates/{l}.png")\n    a.addMatcher(\n        Beholder_Image_Matcher(\n            name=l,\n            layer="gray",\n            filename=str(p),\n            mask_filename=f"./wordtrip_templates/{l}_mask.png",\n            threshhold=0.9,\n        )\n    )'

In [9]:
def wordWorks(word, found_letters):
    found_letters_t = copy(found_letters)
    for letter in word:
        if letter in found_letters_t:
            found_letters_t.remove(letter)
        else:
            return False
    return True

In [10]:
def generateMoves(myword, matches):
    mymatches = copy(matches)
    moves = []
    for letter in myword:
        if letter in mymatches:
            if len(mymatches[letter]) > 0:
                m = mymatches[letter].pop(0)
                moves.append(m.center)

    return moves

In [11]:
from pathlib import Path

import hunspell

hobj = hunspell.HunSpell(
    Path("~/en_US.dic").expanduser(), Path("~/en_US.aff").expanduser()
)


def FindWords(found_letters, spellcheck=False):
    matching_words = []
    for word in tqdm(words):
        if len(word) < min_len or len(word) > max_len:
            continue
        if wordWorks(word, found_letters):
            matching_words.append(word)
    if spellcheck:
        matching_words = list(
            filter(None, [i if hobj.spell(i) else None for i in matching_words])
        )
    matching_words = list(sorted(matching_words, key=len, reverse=True))
    return matching_words

In [12]:
min_len = 3
max_len = 6
eq_len = None

In [13]:
def gsColor(layer, x, y, correct=None):
    color_data = layer.data[y][x]
    if correct is None:
        return color_data
    for index, element in enumerate(color_data):
        if abs(color_data[index] - correct[index]) > 10:
            print("faile on ", abs(color_data[index] - correct[index]))
            return False
    return True

In [14]:
def get_color_distance_fast(color_1, color_2):
    dist = abs(color_1[0] - color_2[0])
    dist += abs(color_1[1] - color_2[1])
    dist += abs(color_1[2] - color_2[2])
    return dist

# Manual matcher setup 

In [15]:
matches = a.findMatches()
print(matches)
found_letters = []
for l in matches:
    for _ in range(0, len(matches[l])):
        found_letters.append(l)
print(found_letters)

matching_words = []
for word in tqdm(words):
    if len(word) < min_len or len(word) > max_len:
        continue
    if wordWorks(word, found_letters):
        matching_words.append(word)
matching_words = list(sorted(matching_words, key=len, reverse=True))
olen = 0
for word in matching_words:
    if len(word) != olen:
        print()
        olen = len(word)
    print(word, end=":")
    print(generateMoves(word, matches))

AddMatch A (648, 2303)
AddMatch C (384, 2139)
AddMatch C (381, 2135)
AddMatch O (940, 1815)
AddMatch T (661, 1650)
{'A': [<Libs.Beholder.BeholderMatch object at 0x7f56ee0cbee0>], 'C': [<Libs.Beholder.BeholderMatch object at 0x7f56ee0cbe80>, <Libs.Beholder.BeholderMatch object at 0x7f56ee0cbf40>], 'O': [<Libs.Beholder.BeholderMatch object at 0x7f56ee0cbfd0>], 'T': [<Libs.Beholder.BeholderMatch object at 0x7f56ed308040>]}
['A', 'C', 'C', 'O', 'T']


  0%|          | 0/370105 [00:00<?, ?it/s]


COACT:[(384, 2139), (940, 1815), (648, 2303), (381, 2135), (661, 1650)]

ACCT:[(648, 2303), (384, 2139), (381, 2135), (661, 1650)]
COAT:[(384, 2139), (940, 1815), (648, 2303), (661, 1650)]
COCA:[(384, 2139), (940, 1815), (381, 2135), (648, 2303)]
COCT:[(384, 2139), (940, 1815), (381, 2135), (661, 1650)]
TACO:[(661, 1650), (648, 2303), (384, 2139), (940, 1815)]

ACC:[(648, 2303), (384, 2139), (381, 2135)]
ACT:[(648, 2303), (384, 2139), (661, 1650)]
CAT:[(384, 2139), (648, 2303), (661, 1650)]
COT:[(384, 2139), (940, 1815), (661, 1650)]
CTO:[(384, 2139), (661, 1650), (940, 1815)]
OAT:[(940, 1815), (648, 2303), (661, 1650)]
OCA:[(940, 1815), (384, 2139), (648, 2303)]
OCT:[(940, 1815), (384, 2139), (661, 1650)]
OTC:[(940, 1815), (661, 1650), (384, 2139)]
TAO:[(661, 1650), (648, 2303), (940, 1815)]
TOA:[(661, 1650), (940, 1815), (648, 2303)]


In [ ]:
raise Exception("Stop")

# Monkey Test

In [2]:
with MonkeyRunner() as monkey:
    while True:
        matches = a.findMatches()
        clicked = False

        closeButtones = list(
            filter(
                None, [name if name.startswith("close_") else None for name in matches]
            )
        )
        if len(closeButtones) > 0:
            m = matches[closeButtones[0]][0]
            print(f"clicking {m.matcher.name}")
            c = m.center
            cmd = f"touch down {c[0]} {c[1]}"
            monkey.raw(cmd)
            for _ in range(0, 10):
                for i in range(-2, 2):
                    for y in range(-2, 2):
                        cmd = f"touch move {c[0]+i} {c[1]+y}"
                        monkey.raw(cmd)

            cmd = f"touch up {c[0]} {c[1]}"
            monkey.raw(cmd)
            continue
        if "claim_backpack" in matches:
            m = matches["claim_backpack"][0]
            print(f"clicking {m.matcher.name}")
            c = m.center
            adb(f"shell input tap {c[0]} {c[1]}")
            continue
        found_letters = []
        for l in matches:
            for _ in range(0, len(matches[l])):
                found_letters.append(l)
        print(found_letters)
        if len(found_letters) == 0:
            sleep(3)
            continue
        matching_words = FindWords(found_letters, spellcheck=True)

        plen = 0

        print(matching_words)
        for word in matching_words:
            a.readNextImage()
            if (
                get_color_distance_fast(
                    [74, 57, 58], gsColor(a.layers["image"], 682, 64)
                )
                < 40
            ):
                print(word)
                x = y = 0
                path = generateMoves(word, matches)
                print(path)
                isDown = False
                for x, y in path:
                    if not isDown:
                        cmd = f"touch down {int(x)} {int(y)}"
                        monkey.raw(cmd)
                        isDown = True
                        time.sleep(0.5)
                        continue
                    cmd = f"touch move {int(x)} {int(y)}"
                    monkey.raw(cmd)
                    time.sleep(0.4)
                cmd = f"touch up {int(x)} {int(y)}"
                monkey.raw(cmd)
                sleep(0.6)
            else:
                print("finished i think")
                break
        sleep(3)

NameError: name 'MonkeyRunner' is not defined

# Research


In [ ]:
raise Exception("Stop")

In [ ]:
channels = cv2.split(a.matchers["white_x"].data)
zero_channel = np.zeros_like(channels[0])

In [ ]:
mask = np.array(channels[3])

In [ ]:
matches = a.findMatches()
matches

In [ ]:
a.matchers

In [ ]:
get_color_distance_fast(correct, gsColor(a.layers["image"], 1190, 1588))

In [ ]:
a.matchers["white_x"].show()

In [ ]:
a.layers["gray"].show()

In [ ]:
matches["I"][1].show(a)

In [ ]:
matches["N"][0].center

shell input touchscreen swipe x1,y1, x2,y2

https://github.com/mattwilson1024/android-pattern-unlock/blob/master/unlock.sh

In [ ]:
generateMoves("FLEET", matches)

In [ ]:
a.layers["image"].data[58][58]

In [ ]:
gsColor(a.layers["image"], 682, 64, [74, 57, 58])

In [ ]:
gsColor(a.layers["image"], 1190, 1588)